In [1]:
import torch
import torch.nn.functional as F

from jamba import JambaLMConfig, JambaLM

In [2]:
config = JambaLMConfig(d_model=64, n_layers=4, mlp_size=128, vocab_size=60, attn_layer_period=100, attn_layer_offset=4)
model = JambaLM(config)
model = model.eval()

In [42]:
def gen_no_caching(num_tokens):
    batch_size = 1
    
    input_ids = 1*torch.ones(batch_size, 1, dtype=torch.int64)

    for i in range(num_tokens):
        next_token_logits, _ = model(input_ids[:, 0:i+1])
        probs = F.softmax(next_token_logits[:, [-1]], dim=-1)
        next_token = torch.argmax(probs, dim=-1)

        input_ids = torch.cat([input_ids, next_token], dim=1)

    return input_ids

In [43]:
gen_no_caching(4)

tensor([[1, 1, 1, 1, 1]])

In [44]:
def gen(num_tokens):
    batch_size = 1

    caches = [(None, torch.zeros(batch_size, config.d_inner, config.d_conv-1)) for _ in range(config.n_layers)]
    
    input_ids = 1*torch.ones(batch_size, 1, dtype=torch.int64)

    for i in range(num_tokens):
        next_token_logits, caches = model(input_ids[:, [i]], caches)
        probs = F.softmax(next_token_logits, dim=-1)
        next_token = torch.argmax(probs, dim=-1)

        input_ids = torch.cat([input_ids, next_token], dim=1)

    return input_ids

In [45]:
gen(4)

tensor([[1, 1, 1, 1, 1]])